In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
from scipy import stats

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///../Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base= automap_base()

# reflect the tables
Base.prepare(autoload_with=engine)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
#Determine the columns in the measurement table
inspector=inspect(engine)
columns=inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
#Determine the columns in the station table
inspector=inspect(engine)
columns=inspector.get_columns('station')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
# Create our session (link) from Python to the DB
session=Session(bind=engine)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
recent_date=session.query(Measurement.date).order_by(Measurement.date.desc()).first()

recent_date

In [ ]:
# Find the earliest date in the data set.
earliest_date=session.query(Measurement.date).order_by(Measurement.date).first()

earliest_date

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
year_ago=dt.date(2017, 8, 23)-dt.timedelta(days=365)

year_ago

In [ ]:
# Perform a query to retrieve the data and precipitation scores
precipitation_data=session.query(Measurement.date, Measurement.prcp).\
                    filter(Measurement.date > year_ago)

In [ ]:
# Save the query results as a Pandas DataFrame. Explicitly set the column names
df=pd.DataFrame(precipitation_data, columns=['date','prcp'])

# Rename the columns for clear titles
df=df.rename(columns={"prcp":"precipitation"})

#View the new DataFrame
df.head()

In [ ]:
# Sort the dataframe by date
sorted_df=df.sort_values("date",ascending=True)
sorted_df.set_index('date', inplace=True)

# Use Pandas Plotting with Matplotlib to plot the data
sorted_df.plot(kind="bar",
                        figsize=(6,4),
                        title="Precipitation by Date",
                        xlabel="Date",
                        ylabel="Inches",
                        grid=True,
                        width=20,
                        )
plt.ylim(-.5,7)
plt.locator_params(axis='x', nbins=5)
plt.xticks(rotation=90)

plt.show()

#I used the following sources to help with formatting my plot, specifically labelling the xticks
#https://stackoverflow.com/questions/6682784/reducing-number-of-plot-ticks/49714879#49714879

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data
sorted_df.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number of stations in the dataset
station_count = session.query(Measurement.station).group_by(Measurement.station).count()

station_count

In [ ]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.
station_data_counts = session.query(Measurement.station, func.count(Measurement.station)).\
              group_by(Measurement.station).\
              order_by(func.count(Measurement.station).desc()).all()

station_data_counts
  

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
sel=[func.min(Measurement.tobs),
     func.max(Measurement.tobs),
     func.avg(Measurement.tobs)]

session.query(*sel).\
filter(Measurement.station == 'USC00519281').all()

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
station_year=session.query(Measurement.tobs).\
            filter(Measurement.station == 'USC00519281').\
            filter(Measurement.date > year_ago).all()

station_year_df=pd.DataFrame(station_year,columns=['tobs'])

#View new data frame
station_year_df.head()

In [ ]:
#plot the results as a histogram
plt.hist(station_year_df, bins=12)
plt.title('Frequency of Temperatures at Station USC00519281')
plt.xlabel('Temperature')
plt.ylabel('Frequency')
plt.show()

# Close Session

In [ ]:
# Close Session
session.close()